# How to use `interrupt` for human-in-the-loop workflows

To use an `interrupt` you must pass a checkpointer.

1. Used with checkpointers
1. Used together with `Command` to resume.
2. Interrupt information is available when streaming. If using invoke with human in the loop need to inspect next
3. Cannot be used twice in a node.

An interrupt can be used within a node like this:

```python
async def some_node(state: State):
    ...
    # Any value that we want to surface as part of the interrupt
    value = {"foo": "bar"} 
    answer = interrupt(value)
    ...
```

Graph execution willbe interrupted when the code reaches
the `interrup` function. It can be resumed by passing `Command`.

```python
graph.invoke(Command(resume=some_value)))
```

## Setup

First we need to install the packages required

In [4]:
%%capture --no-stderr
%pip install --quiet -U langgraph

<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>

## Interrupt and resume

Here is a minimal example that shows how to `interrupt` and `resume` a graph consisting of one node.

In [5]:
import uuid
import operator
from typing import TypedDict, Annotated, Optional

from langgraph.graph import StateGraph
from langgraph.constants import START, INTERRUPT
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver

class State(TypedDict):
    """The graph state."""
    foo: str
    human_value: Optional[str]
    """Human value will be updated using an interrupt."""
    
counter = 0

def node(state: State):
    global counter
    counter +=1 
    print(f'> Entered the node: {counter} # of times')
    answer = interrupt(
        # This value will be sent to the client
        # as part of the interrupt inforamtion.
        'what is your age?'
    )
    print(f'> Received an input from the interrupt: {answer}')
    return {"human_value": answer}

builder = StateGraph(State)
builder.add_node("node", node)
builder.add_edge(START, "node")

# A checkpointer must be enabled for interrupts to work!
graph = builder.compile(checkpointer=MemorySaver())

config = {
    "thread_id": uuid.uuid4(),
}

for chunk in graph.stream({"foo": "abc"}, config):
    print(chunk)

> Entered the node: 1 # of times
{'__interrupt__': (Interrupt(value='what is your age?', resumable=True, ns=['node:d82f393c-4bba-241b-22aa-0cd65a88fd8b'], when='during'),)}


Let's resume graph execution from the given node:

In [6]:
command = Command(resume="some input from a human!!!")

for chunk in graph.stream(Command(resume="some input from a human!!!"), config):
    print(chunk)

> Entered the node: 2 # of times
> Received an input from the interrupt: some input from a human!!!
{'node': {'human_value': 'some input from a human!!!'}}


!!! important "Graph execution resumes at the start of a **node**"

    Graph execution resumes from the **node** where the interrupt was raised rather than from the line where the `interrupt` was raised.

    As a result, you should see  `> Entered the node: 2 # of times`.

## Validation of input with resume

In [127]:
import uuid
import operator
from typing import TypedDict, Annotated, Optional, Literal

from langgraph.graph import StateGraph
from langgraph.graph.state import GraphCommand
from langgraph.constants import START, INTERRUPT
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver


class State(TypedDict):
    """The graph state."""
    foo: str
    human_value: Optional[str]
    """Human value will be updated using an interrupt."""
    

counter = 0

def node(state: State) -> GraphCommand[Literal['node']]:
    global counter
    counter +=1 
    print(f'> Entered the node: {counter} # of times')

    answer = interrupt(
        "What is your age?"
    )

    if not isinstance(answer, int) or answer < 0:
        return GraphCommand(goto="node")

    return GraphCommand(update={"human_value": answer})
        

builder = StateGraph(State)
builder.add_node("node", node)
builder.add_edge(START, "node")

# A checkpointer must be enabled for interrupts to work!
graph = builder.compile(checkpointer=MemorySaver())

config = {
    "thread_id": uuid.uuid4(),
}

for chunk in graph.stream({"foo": "abc"}, config):
    print(chunk)

> Entered the node: 1 # of times
{'__interrupt__': (Interrupt(value='What is your age?', resumable=True, ns=['node:a1c64b81-7957-8c3c-7728-e73cb81837d5'], when='during'),)}


Let's resume with a bad input

In [128]:
bad_input = -20 # Negative number!
for chunk in graph.stream(Command(resume=bad_input), config):
    print(chunk)

> Entered the node: 2 # of times
{'node': None}
> Entered the node: 3 # of times
{'__interrupt__': (Interrupt(value='What is your age?', resumable=True, ns=['node:995027b2-c8ad-8951-74ff-d9b86fca74bb'], when='during'),)}


In [130]:
ok_input = 5
for chunk in graph.stream(Command(resume=ok_input), config):
    print(chunk)

## Usage with invoke / ainvoke

If you're not using `stream` or `astream`, you will need to explicitly access the state of the graph to get information about the interrupt.

In [102]:
import uuid
import operator
from typing import TypedDict, Annotated, Optional

from langgraph.graph import StateGraph
from langgraph.constants import START, INTERRUPT
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver

class State(TypedDict):
    """The graph state."""
    foo: str
    human_value: Optional[str]
    """Human value will be updated using an interrupt."""
    
counter = 0

def node(state: State):
    global counter
    counter +=1 
    print(f'> Entered the node: {counter} # of times')
    answer = interrupt(
        # This value will be sent to the client
        # as part of the interrupt inforamtion.
        'what is your age?'
    )
    print(f'> Received an input from the interrupt: {answer}')
    return {"human_value": answer}

builder = StateGraph(State)
builder.add_node("node", node)
builder.add_edge(START, "node")

# A checkpointer must be enabled for interrupts to work!
graph = builder.compile(checkpointer=MemorySaver())

config = {
    "thread_id": uuid.uuid4(),
}

for event in graph.invoke({"foo": "abc"}, config):
    print

> Entered the node: 1 # of times
